In [19]:
import pandas as pd
from pybliometrics.scopus import AffiliationRetrieval
from pybliometrics.scopus import AuthorRetrieval
from tqdm import tqdm

pd.set_option('display.max_columns', None)
data = pd.read_csv("./data/ai_research_papers.csv")
data.at[222653, 'author_names'] = 'Dudík, Miroslav;Haghtalab, Nika;Luo, Haipeng;Schapire, Robert E.;Syrgkanis, Vasilis;Vaughan, Jennifer Wortman'
data_exploded = data.assign(author_afids=data['author_afids'].str.split(';'), author_ids=data['author_ids'].str.split(';'), author_names = data['author_names'].str.split(';'))
data_exploded = data_exploded.explode(['author_afids', 'author_names', 'author_ids'])
data_exploded['author_rank'] = data_exploded.groupby('eid').cumcount() + 1
data_exploded = data_exploded[['eid', 'year', 'origin_ref','author_count', 'author_names', 'author_ids', 'author_afids', 'author_rank', 'citedby_count']]

data_affil = pd.read_csv("./data/ai_papers_affiliations.csv")
data_affil_app = pd.read_csv("./data/ai_papers_affiliations_append.csv")
data_affil_app = data_affil_app[['afid', 'affiliation_name', 'org_type', 'city', 'state', 'country']].drop_duplicates(subset = 'afid')
data_affil_app['afid'] = data_affil_app['afid'].astype(str)
data_fin_1 = pd.merge(data_exploded, data_affil, how = 'inner', left_on = 'author_afids', right_on = 'afid')
data_fin_2 = pd.merge(data_exploded, data_affil_app, how = 'inner', left_on = 'author_afids', right_on = 'afid')
data_fin = pd.concat([data_fin_1, data_fin_2], ignore_index= True)
data_fin = data_fin.sort_values(['year', 'eid', 'author_rank']).drop_duplicates(subset = ['eid', 'author_ids'])

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,19,21,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
data_fin.loc[data_fin['afid'].isna()]

,eid,year,origin_ref,author_count,author_names,author_ids,author_afids,author_rank,citedby_count,afid,affiliation_name,org_type,city,state,country
98,2-s2.0-0032597866,1999,ICCV Computer Vision,3.0,"Fitzgibbon, Andrew W.",56355070900,,2,119,NaN,NaN,NaN,NaN,NaN,NaN
99,2-s2.0-0032597866,1999,ICCV Computer Vision,3.0,"Zisserman, Andrew",7006619672,,3,119,NaN,NaN,NaN,NaN,NaN,NaN
294,2-s2.0-0038601527,2003,ICSE Software Engineering,5.0,"Jha, Somesh",7202728236,,4,194,NaN,NaN,NaN,NaN,NaN,NaN
644,2-s2.0-58849103803,2007,IJCAI Artificial Intelligence,8.0,"Chandrasekaran, Dhanalakshmi",56357197100,,8,39,NaN,NaN,NaN,NaN,NaN,NaN
826,2-s2.0-85076898936,2019,OSDI Operating Systems,8.0,"Voelker, Geoffrey M.",7003306507,,7,123,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995818,2-s2.0-57349195168,2008,PLDI Programming Languages,3.0,"Gu, Yuan Xiang",57220911033,60078264,2,19,NaN,NaN,NaN,NaN,NaN,NaN
995819,2-s2.0-57349195168,2008,PLDI Programming Languages,3.0,"Johnson, Harold",22734445500,60078264,3,19,NaN,NaN,NaN,NaN,NaN,NaN
995939,2-s2.0-84949007197,1996,PLDI Programming Languages,3.0,"Sefika, Mohlalefi",6507802631,60016910,1,5,NaN,NaN,NaN,NaN,NaN,NaN
995958,2-s2.0-24344499751,1996,PLDI Programming Languages,3.0,"Ecklund, Earl F.",7004593584,100740952,1,10,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

pd.set_option('display.max_rows', None)
data = pd.read_csv("./data/ai_research_papers.csv")
dataset = []
for rows in data.iterrows() :
    #affiliation_country
    #affilname
    af = rows[1]['affiliation_country']
    af = set(af.split(';'))
    dataset.append(list(af))
len(dataset)

# 데이터를 원-핫 인코딩 형태로 변환
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.iloc[:,1:]
frequent_itemsets = apriori(df, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

rules['anc_length'] = rules['antecedents'].apply(lambda x: len(x))
rules['cons_length'] = rules['consequents'].apply(lambda x: len(x))

/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more info

In [8]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
nodes = frequent_itemsets[ (frequent_itemsets['length'] == 1) ].sort_values('support', ascending = False)

In [9]:
edges = rules[['antecedents', 'consequents', 'confidence']].loc[ (rules['anc_length'] == 1) & (rules['cons_length'] == 1)  ].sort_values('confidence', ascending=False)